<a href="https://colab.research.google.com/github/daliaydom/Tarea2_AprendizajeProfundo/blob/main/T2Ejercicio1Convolucion.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Ejercicio 1: Operación de convolución

Dalia Yvette Domínguez Jiménez



In [ ]:
import matplotlib.pyplot as plt
import numpy as np
from sklearn.datasets import load_digits
import requests
from io import BytesIO
from PIL import Image

## Cargamos una imagen a color

In [ ]:
url ='https://raw.githubusercontent.com/daliaydom/Tarea2_AprendizajeProfundo/main/Lena.png'
page = requests.get(url)
Image.open(BytesIO(page.content))

## La convertimos en un arreglo y normalizamos

In [ ]:
img = np.array(Image.open(BytesIO(page.content)))
img = img / img.max()
img.shape

In [ ]:
img.dtype

## Convolución y correlación cruzada
Ahora consideremos las operación de convolución entre una imagen $I$ y un filtro $W$, la cual está definida por

$$
A_{i,j} = (\mathbf{I} * \mathbf{W})_{i,j} = \sum_m \sum_n I_{m, n} W_{i - m, j - n}
$$

La convolución es commutativa, por lo tanto 

$$
A_{i,j} = (\mathbf{W} * \mathbf{I})_{i,j} = \sum_m \sum_n I_{i - m, j - n} W_{m,n}
$$

En lugar de la convolución, frecuentemente se ocupa la operación de correlación cruzada para llevar a cabo las capas convolucionales. Esta operación es similar a la convolución pero sin voltear el filtro (por lo que pierde la propiedad de conmutatividad) y está dada por

$$
A_{i,j} = (\mathbf{W} * \mathbf{I})_{i,j} = \sum_m \sum_n I_{i + m, j + n} W_{m,n} 
$$

El resultado de estas operaciones es el mapa de activaciones $A(i,j)$. 
Como se muestra en la siguiente imagen, esta operación se extiende para 3 canales


![SNOWFALL](https://raw.githubusercontent.com/daliaydom/Tarea2_AprendizajeProfundo/main/CONV3D.png)

In [ ]:
def conv3d(I, W, b, stride = 1):
  h_s = int(np.floor((I.shape[0] - W.shape[0]) / stride)) + 1
  w_s = int(np.floor((I.shape[1] - W.shape[1]) / stride)) + 1
  a = np.zeros((h_s, w_s))
  for i in range(h_s):
    for j in range(w_s):
      I_m = I[i * stride:i * stride + W.shape[0], j * stride:j * stride + W.shape[1],:]
      a[i, j] = (I_m * W).sum() + b                
  return a

## Filtro

Definamos un filtro de tres canales

In [ ]:
N= 20
filter1 = np.zeros([N,N,3])
filter1[:,:,0] = np.identity(N)
filter1[:,:,1] = np.identity(N)
filter1[:,:,2] = np.identity(N)
plt.imshow(filter1)

Aplicando las operaciones de correlación cruzada y convolución

In [ ]:
from scipy import signal
ccorr = signal.correlate(img, filter1, mode = 'valid')
ccorr2 = signal.correlate(filter1, img, mode = 'valid') 
conv = signal.convolve(img, filter1, mode = 'valid') 
conv2 = signal.convolve(filter1, img, mode = 'valid') 
a = conv3d(img, filter1, 0)
fig, axs = plt.subplots(1, 6, figsize=(10, 5))
axs[0].imshow(img, cmap = 'gray') 
axs[1].imshow(np.squeeze(ccorr), cmap = 'gray') 
axs[2].imshow(np.squeeze(ccorr2), cmap = 'gray') 
axs[3].imshow(np.squeeze(conv), cmap = 'gray') 
axs[4].imshow(np.squeeze(conv2), cmap = 'gray') 
axs[5].imshow(a, cmap = 'gray') 
plt.show()

Observa que el resultado de la correlación cruzada de la imagen con el filtro es diferente al del filtro con la imagen. En contraste, la convolución es conmutativa y produce el mismo resultado en ambos casos.



Comparamos los valores de los resultados de la convolución que definimos y la función utilizada

In [ ]:
a[:5,:5]

In [ ]:
ccorr[:5,:5,0]